Forecast stock close price for Algonquin Power & Utilities Corp.(ticker:AQN.TO) using Deep Learning\
Data source: Yahoo Finance\
Models built by: Gary Sampson, DatumSam AI

In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Download Algonquin Power & Utilities Corp. and S&P/TSX composite index from January 1,2010 to October 28, 2022
df = yf.download(['AQN.TO', '^GSPTSE'], start='2010-01-01', end='2022-10-28', 
                    progress=False, auto_adjust=True)
df1 = df['Close']
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3217 entries, 2010-01-04 to 2022-10-27
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   AQN.TO   3217 non-null   float64
 1   ^GSPTSE  3217 non-null   float64
dtypes: float64(2)
memory usage: 75.4 KB


In [3]:
# make a copy of dataset df1
df2 = df1.copy()

In [4]:
# Original dataset did not have weekend and holiday data. Impute with forward fill data to produce timeseries dataset 
Date_new = pd.date_range(start='2010-01-04', end='2022-10-28', freq='D')
df2 = df2.reindex(Date_new, method='ffill' )
df2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4681 entries, 2010-01-04 to 2022-10-28
Freq: D
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   AQN.TO   4681 non-null   float64
 1   ^GSPTSE  4681 non-null   float64
dtypes: float64(2)
memory usage: 109.7 KB


### Forecast AQN.TO stock price using a multivariate time series

In [5]:
df2_multivar = df2

In [6]:
# Create utilities functions to prepare train and validation data
import tensorflow as tf

def to_windows(dataset, length):
    dataset = dataset.window(length, shift=1, drop_remainder=True)
    return dataset.flat_map(lambda window_ds: window_ds.batch(length))

In [7]:
# For multivariate time series,target_col=0 for AQN.TO
def to_seq2seq_dataset_multi(series, seq_length=30, ahead=7, target_col=0,
                       batch_size=32, shuffle=False, seed=None): 
    ds = to_windows(tf.data.Dataset.from_tensor_slices(series), ahead + 1)
    ds = to_windows(ds, seq_length).map(lambda S: (S[:, 0], S[:, 1:, 0]))
    if shuffle:
        ds = ds.shuffle(10 * batch_size, seed=seed)
    return ds.batch(batch_size)

In [8]:
# Split the multivariate dataset into train, validate and test sets
df_multi_train = df2_multivar["2010-01-04":"2019-12-31"]
df_multi_valid = df2_multivar["2021-01-01":"2022-09-01"]
df_multi_test = df2_multivar["2022-09-01":]

In [9]:
# create normalization layer
norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(df_multi_train)
multi_train_scaled = norm_layer(df_multi_train)
multi_valid_scaled = norm_layer(df_multi_valid)

In [10]:
multi_train = to_seq2seq_dataset_multi(multi_train_scaled, shuffle=True, seed=42)
multi_valid = to_seq2seq_dataset_multi(multi_valid_scaled)

In [11]:
# Build gru model with best hyperparameters from previous file(Forecast Canadian stock rev1)
tf.random.set_seed(42)# to ensure reproductibility

gru_multi = tf.keras.models.Sequential([
    tf.keras.layers.GRU(29, return_sequences=True, input_shape=[None,2]),
    tf.keras.layers.Dense(7) 
])

opt = tf.keras.optimizers.Adam(learning_rate=0.0001676)
gru_multi.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=20, restore_best_weights=True)
history_gru_multi = gru_multi.fit(multi_train, validation_data=multi_valid, epochs=300,
                                  callbacks=[early_stopping_cb])

Epoch 1/300
113/113 [==============================] - 9s 63ms/step - loss: 0.3774 - mae: 0.7683 - val_loss: 2.0483 - val_mae: 2.5483
Epoch 2/300
113/113 [==============================] - 6s 53ms/step - loss: 0.2283 - mae: 0.5760 - val_loss: 1.4138 - val_mae: 1.9106
Epoch 3/300
113/113 [==============================] - 2s 21ms/step - loss: 0.0990 - mae: 0.3541 - val_loss: 0.8852 - val_mae: 1.3646
Epoch 4/300
113/113 [==============================] - 3s 23ms/step - loss: 0.0510 - mae: 0.2366 - val_loss: 0.6793 - val_mae: 1.1533
Epoch 5/300
113/113 [==============================] - 2s 21ms/step - loss: 0.0393 - mae: 0.2031 - val_loss: 0.5961 - val_mae: 1.0677
Epoch 6/300
113/113 [==============================] - 2s 20ms/step - loss: 0.0330 - mae: 0.1806 - val_loss: 0.5365 - val_mae: 1.0022
Epoch 7/300
113/113 [==============================] - 2s 21ms/step - loss: 0.0284 - mae: 0.1620 - val_loss: 0.4912 - val_mae: 0.9504
Epoch 8/300
113/113 [==============================] - 2s 19ms

Epoch 62/300
113/113 [==============================] - 2s 18ms/step - loss: 0.0011 - mae: 0.0310 - val_loss: 0.0280 - val_mae: 0.2030
Epoch 63/300
113/113 [==============================] - 2s 21ms/step - loss: 0.0010 - mae: 0.0308 - val_loss: 0.0275 - val_mae: 0.2016
Epoch 64/300
113/113 [==============================] - 2s 19ms/step - loss: 0.0010 - mae: 0.0304 - val_loss: 0.0287 - val_mae: 0.2070
Epoch 65/300
113/113 [==============================] - 2s 20ms/step - loss: 0.0010 - mae: 0.0304 - val_loss: 0.0277 - val_mae: 0.2029
Epoch 66/300
113/113 [==============================] - 2s 18ms/step - loss: 9.8470e-04 - mae: 0.0301 - val_loss: 0.0276 - val_mae: 0.2027
Epoch 67/300
113/113 [==============================] - 3s 23ms/step - loss: 9.6171e-04 - mae: 0.0297 - val_loss: 0.0280 - val_mae: 0.2048
Epoch 68/300
113/113 [==============================] - 3s 22ms/step - loss: 9.5014e-04 - mae: 0.0297 - val_loss: 0.0275 - val_mae: 0.2025
Epoch 69/300
113/113 [=====================

Epoch 121/300
113/113 [==============================] - 4s 31ms/step - loss: 7.1342e-04 - mae: 0.0253 - val_loss: 0.0160 - val_mae: 0.1434
Epoch 122/300
113/113 [==============================] - 3s 26ms/step - loss: 7.1753e-04 - mae: 0.0255 - val_loss: 0.0153 - val_mae: 0.1393
Epoch 123/300
113/113 [==============================] - 3s 22ms/step - loss: 7.1674e-04 - mae: 0.0255 - val_loss: 0.0153 - val_mae: 0.1393
Epoch 124/300
113/113 [==============================] - 3s 29ms/step - loss: 7.1567e-04 - mae: 0.0254 - val_loss: 0.0158 - val_mae: 0.1425
Epoch 125/300
113/113 [==============================] - 3s 25ms/step - loss: 7.1560e-04 - mae: 0.0254 - val_loss: 0.0150 - val_mae: 0.1375
Epoch 126/300
113/113 [==============================] - 3s 26ms/step - loss: 7.1492e-04 - mae: 0.0253 - val_loss: 0.0154 - val_mae: 0.1400
Epoch 127/300
113/113 [==============================] - 3s 27ms/step - loss: 7.1598e-04 - mae: 0.0254 - val_loss: 0.0152 - val_mae: 0.1381
Epoch 128/300
113/11

113/113 [==============================] - 3s 30ms/step - loss: 7.0217e-04 - mae: 0.0249 - val_loss: 0.0121 - val_mae: 0.1190
Epoch 180/300
113/113 [==============================] - 4s 30ms/step - loss: 7.0067e-04 - mae: 0.0249 - val_loss: 0.0124 - val_mae: 0.1208
Epoch 181/300
113/113 [==============================] - 4s 39ms/step - loss: 7.0306e-04 - mae: 0.0250 - val_loss: 0.0120 - val_mae: 0.1182
Epoch 182/300
113/113 [==============================] - 4s 36ms/step - loss: 7.0329e-04 - mae: 0.0250 - val_loss: 0.0119 - val_mae: 0.1174
Epoch 183/300
113/113 [==============================] - 3s 29ms/step - loss: 7.0464e-04 - mae: 0.0250 - val_loss: 0.0120 - val_mae: 0.1175
Epoch 184/300
113/113 [==============================] - 4s 34ms/step - loss: 7.0639e-04 - mae: 0.0252 - val_loss: 0.0123 - val_mae: 0.1201
Epoch 185/300
113/113 [==============================] - 3s 26ms/step - loss: 7.0414e-04 - mae: 0.0251 - val_loss: 0.0120 - val_mae: 0.1183
Epoch 186/300
113/113 [===========

113/113 [==============================] - 3s 27ms/step - loss: 6.9527e-04 - mae: 0.0248 - val_loss: 0.0107 - val_mae: 0.1093
Epoch 238/300
113/113 [==============================] - 3s 25ms/step - loss: 6.9998e-04 - mae: 0.0250 - val_loss: 0.0104 - val_mae: 0.1065
Epoch 239/300
113/113 [==============================] - 3s 27ms/step - loss: 6.9677e-04 - mae: 0.0249 - val_loss: 0.0108 - val_mae: 0.1103
Epoch 240/300
113/113 [==============================] - 4s 38ms/step - loss: 6.9829e-04 - mae: 0.0249 - val_loss: 0.0104 - val_mae: 0.1065
Epoch 241/300
113/113 [==============================] - 4s 31ms/step - loss: 6.9446e-04 - mae: 0.0248 - val_loss: 0.0107 - val_mae: 0.1089
Epoch 242/300
113/113 [==============================] - 3s 26ms/step - loss: 6.9880e-04 - mae: 0.0250 - val_loss: 0.0108 - val_mae: 0.1102
Epoch 243/300
113/113 [==============================] - 3s 28ms/step - loss: 6.9850e-04 - mae: 0.0250 - val_loss: 0.0102 - val_mae: 0.1055
Epoch 244/300
113/113 [===========

In [12]:
# Evaluate best GRU multivariate model
multi_test_scaled = norm_layer(df_multi_test)
multi_test = to_seq2seq_dataset_multi(multi_test_scaled)
gru_multi.evaluate(multi_test)

1/1 [==============================] - 0s 130ms/step - loss: 0.0235 - mae: 0.1724


[0.023482901975512505, 0.17243359982967377]

In [13]:
# Build lstm model with best hyperparameters from previous file(Forecast Canadian stock rev1)
tf.keras.backend.clear_session()

lstm_multi = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(25, return_sequences=True, input_shape=[None,2]),
    tf.keras.layers.LSTM(25, return_sequences=True),
    tf.keras.layers.LSTM(25, return_sequences=True),
    tf.keras.layers.Dense(7) 
])

opt = tf.keras.optimizers.Adam(learning_rate=0.0009245)
lstm_multi.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=20, restore_best_weights=True)
history_lstm_multi = lstm_multi.fit(multi_train, validation_data=multi_valid, epochs=300,
                                  callbacks=[early_stopping_cb])

Epoch 1/300
113/113 [==============================] - 11s 51ms/step - loss: 0.1932 - mae: 0.4974 - val_loss: 0.9270 - val_mae: 1.4253
Epoch 2/300
113/113 [==============================] - 4s 35ms/step - loss: 0.1420 - mae: 0.4164 - val_loss: 0.9330 - val_mae: 1.4323
Epoch 3/300
113/113 [==============================] - 4s 34ms/step - loss: 0.1016 - mae: 0.3521 - val_loss: 0.6727 - val_mae: 1.1603
Epoch 4/300
113/113 [==============================] - 4s 34ms/step - loss: 0.0670 - mae: 0.2710 - val_loss: 0.4788 - val_mae: 0.9360
Epoch 5/300
113/113 [==============================] - 4s 34ms/step - loss: 0.0453 - mae: 0.2084 - val_loss: 0.4135 - val_mae: 0.8609
Epoch 6/300
113/113 [==============================] - 4s 36ms/step - loss: 0.0356 - mae: 0.1828 - val_loss: 0.3525 - val_mae: 0.7807
Epoch 7/300
113/113 [==============================] - 4s 36ms/step - loss: 0.0274 - mae: 0.1535 - val_loss: 0.3173 - val_mae: 0.7351
Epoch 8/300
113/113 [==============================] - 4s 36m

Epoch 61/300
113/113 [==============================] - 4s 36ms/step - loss: 7.7841e-04 - mae: 0.0279 - val_loss: 0.0198 - val_mae: 0.1562
Epoch 62/300
113/113 [==============================] - 4s 36ms/step - loss: 7.4367e-04 - mae: 0.0264 - val_loss: 0.0187 - val_mae: 0.1475
Epoch 63/300
113/113 [==============================] - 4s 37ms/step - loss: 7.4396e-04 - mae: 0.0264 - val_loss: 0.0184 - val_mae: 0.1484
Epoch 64/300
113/113 [==============================] - 5s 47ms/step - loss: 7.3519e-04 - mae: 0.0260 - val_loss: 0.0217 - val_mae: 0.1680
Epoch 65/300
113/113 [==============================] - 5s 42ms/step - loss: 7.3039e-04 - mae: 0.0260 - val_loss: 0.0185 - val_mae: 0.1492
Epoch 66/300
113/113 [==============================] - 5s 44ms/step - loss: 7.6110e-04 - mae: 0.0270 - val_loss: 0.0208 - val_mae: 0.1631
Epoch 67/300
113/113 [==============================] - 5s 47ms/step - loss: 7.4878e-04 - mae: 0.0269 - val_loss: 0.0172 - val_mae: 0.1421
Epoch 68/300
113/113 [=====

In [14]:
# Evaluate best LSTM multivariate model
lstm_multi.evaluate(multi_test)

1/1 [==============================] - 0s 153ms/step - loss: 0.0277 - mae: 0.1884


[0.027728430926799774, 0.1884171962738037]

In [15]:
# using AQN.TO stock close date for 30 days in Sept 2022 to predict stock price from Oct 1 - 7,2022
best_model = tf.keras.Sequential([tf.keras.layers.Reshape((30, 2), input_shape=(None,2)),norm_layer,lstm_multi])
X_new =  df_multi_test[:30].to_numpy().reshape(1, df_multi_test[:30].shape[0], df_multi_test[:30].shape[1]) # one unscaled instance
y_pred_best_model = best_model.predict(X_new)[0,-1] # preprocesses the data and makes predictions
print('AQN.TO stock price Oct 3 - 7,2022:', df_multi_test["2022-10-03":"2022-10-07"]['AQN.TO'])
print('y_pred:', y_pred_best_model[-5:])

1/1 [==============================] - 1s 1s/step
AQN.TO stock price Oct 3 - 7,2022: 2022-10-03    15.161493
2022-10-04    15.424071
2022-10-05    15.025341
2022-10-06    14.480734
2022-10-07    14.334856
Freq: D, Name: AQN.TO, dtype: float64
y_pred: [2.3310995 2.3535476 2.3744514 2.4579985 2.429693 ]


In [16]:
# de-nornalise the prediction from Oct 3 - 7, 2022
multi_train_mean = df_multi_train.mean()
multi_train_std = df_multi_train.std()
print('y_predict:',((y_pred_best_model[-5:]*multi_train_std[0]) + multi_train_mean[0]))

y_predict: [15.083431 15.162645 15.23641  15.531231 15.431347]


In [17]:
# 5 day Mean Absolute error for AQN.to close stock prices from October 3 October 7,2022 using lstm_multi model

err_multi = tf.keras.metrics.MeanAbsoluteError()
err_multi.update_state(df_multi_test["2022-10-03":"2022-10-07"]['AQN.TO'], ((y_pred_best_model[-5:]*multi_train_std[0]) + multi_train_mean[0]))
print("Mean Absolut Error for AQN.to close stock prices from October 3 October 7, 2022:Can$",err_multi.result().numpy())

Mean Absolut Error for AQN.to close stock prices from October 3 October 7, 2022:Can$ 0.539509


In [18]:
# Save the best RNN model trained on multiivariate dataset
from pathlib import Path

model_name = "best_multivariate_rnn"
model_version = "0003"
model_path = Path(model_name) / model_version
best_model.save(model_path, save_format="tf")

INFO:tensorflow:Assets written to: best_multivariate_rnn\0003\assets


INFO:tensorflow:Assets written to: best_multivariate_rnn\0003\assets
